In [8]:
import allel
import zarr
# check which version is installed
print(allel.__version__)

1.3.5


In [11]:
import malariagen_data
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")

# DSX amplicon
region = "2R:48,714,427-48,714,714"
samples = ag3.sample_metadata(sample_sets="v3_wild")


In [6]:
ds = ag3.snp_calls(region=region, sample_sets="v3_wild")

In [7]:
ds

<xarray.Dataset>
Dimensions:                             (variants: 288, alleles: 4, samples: 2784, ploidy: 2)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(288,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(288,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(81,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(288, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(288,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(288,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(288,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(288, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(288, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(288, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(288, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(288, 50, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [12]:
import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
import allel

# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})

mask = 'gamb_colu_arab'

loc_pass = ds[f"variant_filter_pass_{mask}"].values
gt = allel.GenotypeDaskArray(ds["call_genotype"][loc_pass].data)


In [13]:
gt.shape

(280, 2784, 2)

In [14]:
with ProgressBar():
    ac = gt.count_alleles(max_allele=3).compute()

[########################################] | 100% Completed |  1min 37.3s


In [15]:
is_variant = ac.is_variant()
ac = ac[is_variant]
ac

<AlleleCountsArray shape=(75, 4) dtype=int32>
5557   11    0    0
5460   16   92    0
5564    4    0    0
...
5564    4    0    0
5567    0    0    1
5559    9    0    0

In [16]:
# get positions
pos = ds['variant_position'][loc_pass][is_variant].compute()
pos

<xarray.DataArray 'variant_position' (variants: 75)>
array([48714441, 48714445, 48714447, 48714450, 48714453, 48714454,
       48714457, 48714458, 48714467, 48714468, 48714472, 48714473,
       48714474, 48714476, 48714481, 48714483, 48714486, 48714491,
       48714493, 48714494, 48714498, 48714499, 48714500, 48714502,
       48714503, 48714505, 48714507, 48714508, 48714510, 48714512,
       48714513, 48714515, 48714516, 48714518, 48714521, 48714524,
       48714527, 48714533, 48714535, 48714538, 48714542, 48714544,
       48714546, 48714552, 48714553, 48714563, 48714566, 48714578,
       48714581, 48714592, 48714617, 48714620, 48714624, 48714626,
       48714641, 48714652, 48714664, 48714670, 48714671, 48714672,
       48714675, 48714680, 48714681, 48714685, 48714689, 48714692,
       48714695, 48714698, 48714699, 48714700, 48714701, 48714708,
       48714709, 48714711, 48714714], dtype=int32)
Coordinates:
    variant_position  (variants) int32 48714441 48714445 ... 48714711 48714714
    variant_contig    (variants) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
Dimensions without coordinates: variants

In [17]:
# ds = ag3.snp_calls(region=region, sample_sets="v3_wild")
cohorts_ = dict(
    [(f"{c}|{s}", list(df.index)) for (c, s), df in samples.groupby(['country', 'aim_species'])]
     )

In [32]:
cohorts_country = dict(
    [(f"{c}", list(df.index)) for c, df in samples.groupby(['country'])]
     )

In [19]:
ac_subpop = {}

for subpop in cohorts_.keys():
    print(subpop,)
    ac_subpop[subpop] = gt.count_alleles(max_allele=3, subpop=cohorts_[subpop]).compute()

Angola|coluzzii
Burkina Faso|arabiensis
Burkina Faso|coluzzii
Burkina Faso|gambiae
Burkina Faso|intermediate_gambiae_coluzzii
Cameroon|arabiensis
Cameroon|coluzzii
Cameroon|gambiae
Central African Republic|coluzzii
Central African Republic|gambiae
Cote d'Ivoire|coluzzii
Democratic Republic of the Congo|gambiae
Equatorial Guinea|gambiae
Gabon|gambiae
Gambia, The|coluzzii
Gambia, The|gambiae
Gambia, The|intermediate_gambiae_coluzzii
Ghana|coluzzii
Ghana|gambiae
Guinea|coluzzii
Guinea|gambiae
Guinea|intermediate_gambiae_coluzzii
Guinea-Bissau|gambiae
Guinea-Bissau|intermediate_gambiae_coluzzii
Kenya|arabiensis
Kenya|gambiae
Kenya|intermediate_gambiae_coluzzii
Malawi|arabiensis
Mali|arabiensis
Mali|coluzzii
Mali|gambiae
Mali|intermediate_gambiae_coluzzii
Mayotte|gambiae
Mozambique|gambiae
Tanzania|arabiensis
Tanzania|gambiae
Tanzania|intermediate_gambiae_coluzzii
Uganda|arabiensis
Uganda|gambiae
Uganda|intermediate_arabiensis_gambiae


In [31]:
import pandas as pd

df_pos = pd.DataFrame(pos, columns=['pos'])
df_ac = pd.DataFrame(ac, columns=['wt_ac', 'alt_1_ac', 'alt_2_ac', 'alt_3_ac'])
df_af = pd.DataFrame(ac.to_frequencies(), columns=['wt_af', 'alt_1_af', 'alt_2_af', 'alt_3_af'])

df = pd.concat([df_pos, df_ac, df_af], axis=1)
df_bf.to_csv('dsx_amplicon_Ag1000g_phase3_ac_af.csv')

,pos,wt_ac,alt_1_ac,alt_2_ac,alt_3_ac,wt_af,alt_1_af,alt_2_af,alt_3_af
0,48714441,5557,11,0,0,0.998024,0.001976,0.000000,0.00000
1,48714445,5460,16,92,0,0.980603,0.002874,0.016523,0.00000
2,48714447,5564,4,0,0,0.999282,0.000718,0.000000,0.00000
3,48714450,5567,0,0,1,0.999820,0.000000,0.000000,0.00018
4,48714453,5526,42,0,0,0.992457,0.007543,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...
70,48714701,5565,3,0,0,0.999461,0.000539,0.000000,0.00000
71,48714708,5566,0,2,0,0.999641,0.000000,0.000359,0.00000
72,48714709,5564,4,0,0,0.999282,0.000718,0.000000,0.00000
73,48714711,5567,0,0,1,0.999820,0.000000,0.000000,0.00018


In [36]:
# BF
ac_bf = gt.count_alleles(max_allele=3, subpop=cohorts_country['Burkina Faso']).compute()
ac_bf = ac_bf[is_variant]

In [37]:
df_bf_ac = pd.DataFrame(ac_bf, columns=['wt_ac', 'alt_1_ac', 'alt_2_ac', 'alt_3_ac'])
df_bf_af = pd.DataFrame(ac_bf.to_frequencies(), columns=['wt_af', 'alt_1_af', 'alt_2_af', 'alt_3_af'])

df_bf = pd.concat([df_pos, df_bf_ac, df_bf_af], axis=1)
df_bf.to_csv('dsx_amplicon_Ag1000g_phase3_ac_af.burkina_faso.csv')

,pos,wt_ac,alt_1_ac,alt_2_ac,alt_3_ac,wt_af,alt_1_af,alt_2_af,alt_3_af
0,48714441,592,0,0,0,1.000000,0.000000,0.000000,0.0
1,48714445,586,2,4,0,0.989865,0.003378,0.006757,0.0
2,48714447,592,0,0,0,1.000000,0.000000,0.000000,0.0
3,48714450,592,0,0,0,1.000000,0.000000,0.000000,0.0
4,48714453,575,17,0,0,0.971284,0.028716,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
70,48714701,591,1,0,0,0.998311,0.001689,0.000000,0.0
71,48714708,592,0,0,0,1.000000,0.000000,0.000000,0.0
72,48714709,592,0,0,0,1.000000,0.000000,0.000000,0.0
73,48714711,592,0,0,0,1.000000,0.000000,0.000000,0.0
